In [1]:
import pickle
from pathlib import Path
import sys

import pandas as pd
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

from module.mymodule import grid_search_cv
from module.features import pipe_1, pipe_2, pipe_3, pipe_4, pipe_5, pipe_6, pipe_7,\
                     pipe_8, pipe_9, pipe_10, pipe_11, pipe_12, pipe_13, pipe_14,\
                     pipe_15

# モデルのチューニングと訓練を行う

### 特徴量候補を用意してデータセット作成

In [2]:
df = pd.read_csv('./data/train.csv')
df = df.loc[:550, :]
to_pipe ={
            'df': df,
            'split_kwrg': {'test_size': 0.2, 'to_array': True},
            'train_flg': True,
            'retrain': False,
            }
# 特徴量候補を設定
pipe_lines = [
            pipe_1,  # base 
            # pipe_2,  # StSlopeCat
            # pipe_3,  # CholsetMean
            # pipe_4,  # AgeCat
            # pipe_5,  # StSlopeCat CholetMean AgeCat
            # pipe_6,  # RestingBpCat
            # pipe_7,  # OldPeakCat
            # pipe_8,  # RestingBpCat OldPeakCat
            pipe_9,  # Onehot
            pipe_10,  # CholestMean AgeCat Onehot
            pipe_11,  # CholCut
            pipe_12,  # CholCut Onehot,
            pipe_13,  # DropByShap
            pipe_14,  # CholCatDropBy,
            pipe_15  # CholRegression
            ]
data_set = {pipe.__name__: pipe(**to_pipe) for pipe in pipe_lines}

                                Base(pipe_1)                               


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,56,1,155,342,1,150,1,3.0
1,55,0,130,394,0,150,0,0.0
2,47,1,110,0,1,120,1,0.0
3,34,1,115,0,1,154,0,0.2
4,54,0,160,201,0,163,0,0.0


                          Onehot Standard(pipe_9)                          


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,47,1,110,0,1,120,1,0.0,0,0,1,0,0,1,0,0,1,0
3,34,1,115,0,1,154,0,0.2,1,0,0,0,0,1,0,0,0,1
4,54,0,160,201,0,163,0,0.0,0,0,1,0,0,1,0,0,0,1


                CholestMean AgeCat Onehot Standard(pipe_10)                


...

,Age,Sex,Cholesterol,FastingBS,MaxHR,Oldpeak,ChestPainType_ASY,ST_Slope_Flat,ST_Slope_Up
0,56,1,342,1,150,3.0,1,1,0
1,55,0,394,0,150,0.0,0,0,1
2,47,1,0,1,120,0.0,0,1,0
3,34,1,0,1,154,0.2,1,0,1
4,54,0,201,0,163,0.0,0,0,1


                        CholCut DropByShap(pipe_14)                        


,Age,Sex,Cholesterol,FastingBS,MaxHR,Oldpeak,ChestPainType_ASY,ST_Slope_Flat,ST_Slope_Up
0,56,1,342,1,150,3.0,1,1,0
1,55,0,394,0,150,0.0,0,0,1
2,54,0,201,0,163,0.0,0,0,1
3,64,1,335,0,158,0.0,0,0,1
4,65,1,254,0,127,2.8,1,1,0


                          Chol Regression(pipe_15)                         


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,56,1,155,342,1,150,1,3.0,1,0,0,0,0,1,0,0,1,0
1,55,0,130,394,0,150,0,0.0,0,1,0,0,1,0,0,0,0,1
2,47,1,110,230,1,120,1,0.0,0,0,1,0,0,1,0,0,1,0
3,34,1,115,248,1,154,0,0.2,1,0,0,0,0,1,0,0,0,1
4,54,0,160,201,0,163,0,0.0,0,0,1,0,0,1,0,0,0,1


### モデル候補を用意

In [3]:
# XGBoost
xgboost = {'model': XGBClassifier,
           'param_grid': {
                          'max_depth':[3, 5, 7, 9, 15],
                          'learning_rate': [0.05, 0.1, 0.3],
                          'n_estimators': [50, 75, 100, 150],
                          },
           'model_arg': {'random_state': 42, 'early_stopping_rounds': 50}
          }

# Tree
tree = {'model': DecisionTreeClassifier,
           'param_grid': {
                          'max_depth':[1, 2, 3, 4, 5],
                          'min_samples_leaf':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'min_samples_split':[2, 3, 4, 5]
                          },
           'model_arg': {'random_state': 42,}
          }

# SVC
svc = {'model': SVC,
           'param_grid': {
                          'C': [0.01, 0.1, 1.0, 10],
                          'kernel': ['linear', 'rbf'],
                          'probability': [True],
                          },
           'model_arg': {'random_state': 42}
          }

### モデルの訓練

In [4]:
# モデルの候補を設定
model_candidates = [
                    xgboost,
                    tree,
                    svc
                    ]

trained_models = {}  # 訓練したモデルの格納先
for candidate in model_candidates:
    print(candidate['model'].__name__.center(50, '#'))
    models = {}
    for key, pack in data_set.items():
        print(key.center(50))
        models[key] = grid_search_cv(pack, **candidate)
    trained_models[candidate['model'].__name__] = models
    
#モデルを保存
file_name = 'models'
with open(f'./data/{file_name}.pkl', mode='wb') as f:
    pickle.dump(trained_models, f)

##################XGBClassifier###################
                      pipe_1                      
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.929545,0.976744,0.889831,0.931264
test,0.855856,0.916667,0.833333,0.873016


                      pipe_9                      
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.972727,0.966667,0.983051,0.974790
test,0.891892,0.897059,0.924242,0.910448


                     pipe_10                      
-------------------- 評価結果 --------------------


...

,accuracy_score,precision_score,recall_score,f1_score
train,0.900000,0.910256,0.902542,0.906383
test,0.891892,0.921875,0.893939,0.907692


                     pipe_14                      
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.879213,0.876623,0.849057,0.862620
test,0.842697,0.872340,0.836735,0.854167


                     pipe_15                      
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.920455,0.913580,0.940678,0.926931
test,0.909910,0.878378,0.984848,0.928571
